In [ ]:
import pandas as pd
import os
import sys

import yaml
import logging
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from keras.layers import Dense , Dropout , Flatten , Activation , BatchNormalization , Input , Embedding , Dot
from keras.models import Sequential
from keras.utils import to_categorical

from keras.callbacks import EarlyStopping , ModelCheckpoint  , LearningRateScheduler , TensorBoard , CSVLogger
%matplotlib inline


READING CSV

In [ ]:
anime_df = pd.read_csv(
    r"D:\anime_recommend_mlops\archive\animelist.csv",
    low_memory=True,
    nrows=50000000,
    usecols=["user_id", "anime_id", "rating"],
    dtype={'user_id': 'int32', 'anime_id': 'int32', 'rating': 'int8'}
)

In [ ]:
anime_df.head()

In [ ]:
len(anime_df)

DATA PROCESSING

In [ ]:
n_rating = anime_df['user_id'].value_counts().sort_index()

In [ ]:
n_rating

In [ ]:
anime_df = anime_df[anime_df["user_id"].isin(n_rating[n_rating >=400].index)].copy()

In [ ]:
len(anime_df)

In [ ]:
min_rating = min(anime_df['rating'])

In [ ]:
max_rating = max(anime_df['rating'])

In [ ]:
min_rating

In [ ]:
max_rating

In [ ]:
avg_rating = np.mean(anime_df['rating'])

In [ ]:
avg_rating

FEATURE SCALING

In [ ]:
anime_df["rating"] = anime_df["rating"].apply(lambda x: (x-min_rating)/(max_rating-min_rating)).values.astype(float)

In [ ]:
avg_rating = np.mean(anime_df['rating'])

In [ ]:
avg_rating

In [ ]:
anime_df.isna().sum()

In [ ]:
anime_df.duplicated().sum()

In [ ]:
user_ids = anime_df['user_id'].unique().tolist()
print(user_ids)

In [ ]:
user2user_encoded = {x:i for i , x in enumerate(user_ids)}

In [ ]:
user2user_encoded

In [ ]:
user2user_decoded = { i : x for x , i in enumerate(user_ids)}

In [ ]:
user2user_decoded

In [ ]:
anime_df["user"] = anime_df["user_id"].map(user2user_encoded)

In [ ]:
anime_df.head()

In [ ]:
anime_ids = anime_df['anime_id'].unique().tolist()

In [ ]:
anime2anime_encoded = { x : i for i , x in enumerate(anime_ids)}

In [ ]:
# Mapping and optimization combined to save memory and handle out-of-order execution
if 'user' not in anime_df.columns:
    anime_df['user'] = anime_df['user_id'].map(user2user_encoded).astype('int32')
if 'anime' not in anime_df.columns:
    anime_df['anime'] = anime_df['anime_id'].map(anime2anime_encoded).astype('int32')

anime_df['rating'] = anime_df['rating'].astype('float32')
if 'user_id' in anime_df.columns:
    anime_df = anime_df.drop(columns=['user_id', 'anime_id'])
anime_df = anime_df.sample(frac=1 , random_state=43).reset_index(drop=True)

In [ ]:
anime_df["anime"] = anime_df["anime_id"].map(anime2anime_encoded)

In [ ]:
anime_df.head()

In [ ]:
# Mapping and optimization combined to save memory and handle out-of-order execution
if 'user' not in anime_df.columns:
    anime_df['user'] = anime_df['user_id'].map(user2user_encoded).astype('int32')
if 'anime' not in anime_df.columns:
    anime_df['anime'] = anime_df['anime_id'].map(anime2anime_encoded).astype('int32')

anime_df['rating'] = anime_df['rating'].astype('float32')
if 'user_id' in anime_df.columns:
    anime_df = anime_df.drop(columns=['user_id', 'anime_id'])
anime_df = anime_df.sample(frac=1 , random_state=43).reset_index(drop=True)

SPLITING DATA

In [ ]:
X = anime_df[["user","anime"]].values
y = anime_df["rating"]